In [1]:
import os

In [3]:
os.chdir("../")
# 'd:\\End-to-End chicken-diesase-implementation'

In [5]:
import tensorflow as tf


In [12]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [17]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class ModelEvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int
    
    
    

In [18]:
from src.cnnClassifier.constant import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [19]:
class ConfigurationManager:
    
    def __init__(self, configfile = CONFIG_FILE_PATH, paramsfile = PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(configfile)
        self.params = read_yaml(paramsfile)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_validation_config(self) -> ModelEvaluationConfig:
        
        model_eval = ModelEvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data = "data/Chicken-fecal-images",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )
        
        return model_eval
        

In [22]:
class Evaluation:
    def __init__(self, config: ModelEvaluationConfig) -> None:
        self.config = config
        
        
    def _train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )
        
        dataflow_kwargs = dict(
            
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
    
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs,
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._train_valid_generator()
        self.score = model.evaluate(self.valid_generator)
        
    def save_score(self):
        score = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("score.json"), data=score)
        
            
    

In [24]:
try:
    config = ConfigurationManager()
    get_eval = config.get_validation_config()
    evaluations = Evaluation(get_eval)
    evaluations.evaluation()
    evaluations.save_score()
    
except Exception as e:
    raise e

[2023-08-02 00:10:45,952 - INFO - common - yaml file: config\config.yaml loaded successfully]
[2023-08-02 00:10:45,958 - INFO - common - yaml file: params.yaml loaded successfully]
[2023-08-02 00:10:45,963 - INFO - common - Directory created ! artifacts]
Found 78 images belonging to 2 classes.
5/5 [==============================] - 7s 1s/step - loss: 0.3348 - accuracy: 0.8077
[2023-08-02 00:10:54,160 - INFO - common - json file saved at: score.json]
